In [ ]:
# Testuję różne warianty ID zmiennej dla mediany cen
import requests

test_ids = [3787, "3787", "P3787", "p3787"]

for vid in test_ids:
    try:
        r = requests.get(f"https://bdl.stat.gov.pl/api/v1/variables/{vid}", 
                        params={"format": "json", "lang": "pl"}, timeout=10)
        r.raise_for_status()
        meta = r.json()
        print(f"ID={vid:8} ✔ Znaleziono: {meta.get('n1','')} | {meta.get('measureUnitName','')}")
    except Exception as e:
        print(f"ID={vid:8} ✘ Błąd: {str(e)[:60]}")

ID=    3787 ✔ Znaleziono: zameldowania | osoba


In [ ]:
# Szukam zmiennych w podgrupie P3787 (mediana cen mieszkań)
import requests

r = requests.get("https://bdl.stat.gov.pl/api/v1/variables", 
                params={"subject-id": "P3787", "format": "json", "lang": "pl"}, 
                timeout=10)
print("Status:", r.status_code)
if r.status_code == 200:
    data = r.json()
    results = data.get("results", [])
    print(f"\nZnaleziono {len(results)} zmiennych w podgrupie P3787:\n")
    for v in results[:20]:  # pierwsze 20
        print(f"  ID={v.get('id'):8} | {v.get('n1','')} | {v.get('measureUnitName','')}")
else:
    print("Błąd:", r.text[:200])

Status: 200

Znaleziono 10 zmiennych w podgrupie P3787:

  ID=  633677 | ogółem | zł
  ID=  633678 | ogółem | zł
  ID=  633679 | ogółem | zł
  ID=  633680 | ogółem | zł
  ID=  633681 | ogółem | zł
  ID=  633682 | rynek pierwotny | zł
  ID=  633683 | rynek pierwotny | zł
  ID=  633684 | rynek pierwotny | zł
  ID=  633685 | rynek pierwotny | zł
  ID=  633686 | rynek pierwotny | zł


In [ ]:
# Sprawdzam szczegóły zmiennych z rynku pierwotnego (633682-633686)
for vid in [633682, 633683, 633684, 633685, 633686]:
    r = requests.get(f"https://bdl.stat.gov.pl/api/v1/variables/{vid}", 
                    params={"format": "json", "lang": "pl"}, timeout=10)
    if r.status_code == 200:
        meta = r.json()
        print(f"\nID {vid}:")
        print(f"  n1: {meta.get('n1','')}")
        print(f"  n2: {meta.get('n2','')}")
        print(f"  n3: {meta.get('n3','')}")
        print(f"  Jednostka: {meta.get('measureUnitName','')}")


ID 633682:
  n1: rynek pierwotny
  n2: ogółem
  n3: 
  Jednostka: zł

ID 633683:
  n1: rynek pierwotny
  n2: do 40 m2
  n3: 
  Jednostka: zł

ID 633683:
  n1: rynek pierwotny
  n2: do 40 m2
  n3: 
  Jednostka: zł

ID 633684:
  n1: rynek pierwotny
  n2: od 40,1 do 60 m2
  n3: 
  Jednostka: zł

ID 633684:
  n1: rynek pierwotny
  n2: od 40,1 do 60 m2
  n3: 
  Jednostka: zł

ID 633685:
  n1: rynek pierwotny
  n2: od 60,1 do 80 m2
  n3: 
  Jednostka: zł

ID 633685:
  n1: rynek pierwotny
  n2: od 60,1 do 80 m2
  n3: 
  Jednostka: zł

ID 633686:
  n1: rynek pierwotny
  n2: od 80,1 m2
  n3: 
  Jednostka: zł

ID 633686:
  n1: rynek pierwotny
  n2: od 80,1 m2
  n3: 
  Jednostka: zł


In [ ]:
import requests
import pandas as pd
import csv

BASE_URL = "https://bdl.stat.gov.pl/api/v1"

def fetch_all_by_variable(var_id, unit_level=None, years=None, lang="pl", page_size=100):
    """
    Pobiera wszystkie strony wyników z /data/by-variable/{var_id}.
    Zwraca listę obiektów 'results'.
    Bezpiecznie obsługuje nieoczekiwane struktury odpowiedzi.
    """
    page = 0
    all_results = []

    while True:
        params = {
            "page": page,
            "page-size": page_size,
            "format": "json",
            "lang": lang
        }
        if unit_level is not None:
            params["unit-level"] = unit_level
        if years is not None:
            params["year"] = ",".join(map(str, years))

        r = requests.get(f"{BASE_URL}/data/by-variable/{var_id}", params=params, timeout=60)
        r.raise_for_status()
        try:
            data = r.json()
        except ValueError:
            raise RuntimeError("Odpowiedź nie jest JSON")

        raw_results = data.get("results", [])
        if not isinstance(raw_results, list):
            raw_results = [raw_results]
        all_results.extend(raw_results)

        links = data.get("links", {}) or {}
        if links.get("next"):
            page += 1
            continue

        total_pages = data.get("totalPages", None)
        if isinstance(total_pages, int) and page < total_pages - 1:
            page += 1
            continue

        break

    return all_results

def get_variable_meta(var_id, lang="pl"):
    """Pobiera nazwę zmiennej i jednostkę miary z /variables/{id}."""
    r = requests.get(f"{BASE_URL}/variables/{var_id}", params={"lang": lang, "format": "json"}, timeout=60)
    r.raise_for_status()
    meta = r.json()
    name = (meta.get("name") or meta.get("n1") or meta.get("title") or str(var_id)).strip()
    unit = (meta.get("measureUnitName") or meta.get("unit") or meta.get("measureUnit") or "").strip()
    return name, unit

def results_to_long(results):
    """
    Zamienia wyniki API do formatu długiego: Kod (7), Nazwa, Year, Value.
    Obsługuje dwie struktury: lista 'values' lub płaska lista rekordów.
    Odporne na brak spodziewanych kluczy.
    """
    if not isinstance(results, list):
        raise TypeError("results powinno być listą słowników")
    rows = []

    for item in results:
        if not isinstance(item, dict):
            continue
        unit_id = str(item.get("unitId") or item.get("id") or (item.get("unit") or {}).get("id") or "")
        unit_name = (item.get("unitName") or item.get("name") or (item.get("unit") or {}).get("name") or "").strip()
        code7 = unit_id[:7]
        values = item.get("values") or item.get("data") or None

        if isinstance(values, list) and values and isinstance(values[0], dict):
            for v in values:
                year = v.get("year")
                val = v.get("val") if "val" in v else v.get("value")
                if year is None:
                    continue
                rows.append({"Kod": code7, "Nazwa": unit_name, "Year": int(year), "Value": val})
        else:
            year = item.get("year")
            val = item.get("val") if "val" in item else item.get("value")
            if year is None:
                continue
            rows.append({"Kod": code7, "Nazwa": unit_name, "Year": int(year), "Value": val})

    return pd.DataFrame(rows)

def make_wide(df_long, series_prefix, unit_label=""):
    if df_long.empty:
        return pd.DataFrame(columns=["Kod","Nazwa"])
    years = sorted([y for y in df_long["Year"].unique() if pd.notna(y)])
    col_labels = [f"{series_prefix};{y};[{unit_label}]" if unit_label else f"{series_prefix};{y}" for y in years]
    wide = (df_long.pivot_table(index=["Kod","Nazwa"], columns="Year", values="Value", aggfunc="first").reset_index())
    wide.columns = ["Kod","Nazwa"] + col_labels
    return wide

def write_like_bdl_csv(df_wide, out_path):
    import os
    if df_wide.empty:
        print("Ostrzeżenie: df_wide jest pusty – plik nie został utworzony.")
        return
    # Tworzymy folder data jeśli nie istnieje
    out_dir = os.path.dirname(out_path)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir)
    tmp = out_path + ".tmp"
    # POPRAWA: używamy parametru 'lineterminator' zamiast 'line_terminator'
    df_wide.to_csv(tmp, sep=";", index=False, quoting=csv.QUOTE_MINIMAL, lineterminator="\n")
    with open(tmp, "r", encoding="utf-8") as f:
        lines = f.read().splitlines()
    if not lines:
        print("Brak linii w pliku tymczasowym – przerwano.")
        return
    header_fields = lines[0].split(";")
    header_quoted = ";".join([f'"{h}"' for h in header_fields]) + ";"
    new_lines = [header_quoted]
    for ln in lines[1:]:
        if not ln.endswith(";"):
            ln += ";"
        new_lines.append(ln)
    with open(out_path, "w", encoding="utf-8", newline="\n") as f:
        f.write("\n".join(new_lines))
    os.remove(tmp)
    print("Plik zapisany:", out_path)

# =========================
# UŻYCIE (URUCHAMIAJ W NOTEBOOKU)
# =========================
VAR_ID = 633682     # <- Mediana cen m2 mieszkań (rynek pierwotny, ogółem)
UNIT_LEVELS = [0, 2, 5]  # 0=Polska, 2=województwa, 5=powiaty
YEARS = range(2014, 2025)  # 2014–2024

# Pobieramy dane dla wszystkich poziomów
all_results = []
for level in UNIT_LEVELS:
    print(f"Pobieram poziom {level}...")
    results = fetch_all_by_variable(VAR_ID, unit_level=level, years=YEARS)
    all_results.extend(results)
    print(f"  Pobrano {len(results)} jednostek")

print(f'\nRazem pobrano rekordów: {len(all_results)}')
var_name, unit_name = get_variable_meta(VAR_ID)
print('Meta:', var_name, '|', unit_name)
SERIES_PREFIX = "rynek pierwotny;ogółem"
df_long = results_to_long(all_results)
print('df_long shape:', df_long.shape)
df_wide = make_wide(df_long, series_prefix=SERIES_PREFIX, unit_label=unit_name or "zł")
print('df_wide shape:', df_wide.shape)
write_like_bdl_csv(df_wide, f"data/RYNE_{VAR_ID}_CTAB_export.csv")

Pobieram poziom 0...
  Pobrano 1 jednostek
Pobieram poziom 2...
  Pobrano 1 jednostek
Pobieram poziom 2...
  Pobrano 16 jednostek
Pobieram poziom 5...
  Pobrano 16 jednostek
Pobieram poziom 5...
  Pobrano 380 jednostek

Razem pobrano rekordów: 397
  Pobrano 380 jednostek

Razem pobrano rekordów: 397
Meta: rynek pierwotny | zł
df_long shape: (5952, 4)
df_wide shape: (397, 17)
Plik zapisany: data/RYNE_633682_CTAB_export.csv
Meta: rynek pierwotny | zł
df_long shape: (5952, 4)
df_wide shape: (397, 17)
Plik zapisany: data/RYNE_633682_CTAB_export.csv
